In [ ]:
# prompt: install langchain

!pip install langchain
!pip install langchain-google-genai

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
import google.generativeai as genai

In [ ]:
from google.colab import auth
from googleapiclient.discovery import build
from google.auth.transport.requests import Request
import os

auth.authenticate_user()

drive_service = build('drive', 'v3')
#folder_id = '1AUQMMqnypEj8GGRcIiGeWMhHmwLgZZj6'  # arrest

folder_id = '1ObEc85b83DK9FnSDKpeeaUDUMt-4WDdh'  # abuse
results = drive_service.files().list(
    q=f"'{folder_id}' in parents", fields="files(id, name)").execute()

files = results.get('files', [])


In [ ]:
import io
from googleapiclient.http import MediaIoBaseDownload

# Loop through each file in the folder and download it to the abuse folder
for file in files:
    file_id = file['id']
    file_name = file['name']

    # Define the path to save the file
    file_path = os.path.join('/content/abuse', file_name)

    # Request the file from Google Drive
    request = drive_service.files().get_media(fileId=file_id)

    # Open a file handle to save the file
    fh = io.FileIO(file_path, 'wb')
    downloader = MediaIoBaseDownload(fh, request)

    done = False
    while done is False:
        status, done = downloader.next_chunk()
        print(f"Download {file_name} to {file_path} {int(status.progress() * 100)}%.")


In [ ]:
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('gemini_api')


In [ ]:
!pip install -q -U google-generativeai

In [ ]:
import google.generativeai as genai
from google.colab import userdata
genai.configure(api_key=userdata.get('gemini')
)
model = genai.GenerativeModel("gemini-1.5-pro-latest" )


In [ ]:
import os
import time
import google.generativeai as genai

# Folder paths for video files
abuse_folder = '/content/abuse'
arrest_folder = '/content/arrest'

# Collect video files
abuse_videos = [os.path.join(abuse_folder, f) for f in os.listdir(abuse_folder) if f.endswith('.mp4')]
arrest_videos = [os.path.join(arrest_folder, f) for f in os.listdir(arrest_folder) if f.endswith('.mp4')]
all_videos = abuse_videos + arrest_videos

# Output reports folder
reports_folder = '/content/reports'
os.makedirs(reports_folder, exist_ok=True)  # Create the reports folder if it doesn't exist

# Define the prompt
prompt_template = """
Incident Description: Summarize the events in the video, describing the fight or violent behavior in detail and provide the following:

1. Incident Summary: Key moments, timeline, and location.
2. Involved Parties: Gender, age, clothing, actions, and features of each person.
3. Incident Type: Main action (e.g., Abuse, Arrest) and secondary actions.
4. Injuries: List injuries for each person and damage to property.
5. Time and Date: When did the incident happen?
6. Outcome: Resolution, arrests, and follow-up actions.
"""

# Process videos
for video_path in all_videos:
    video_name = os.path.basename(video_path)
    print(f"Processing video: {video_name}")

    try:
        # Upload video file
        video_file = genai.upload_file(path=video_path)
        print(f"Uploaded: {video_file.uri}")

        # Wait for processing to complete
        while video_file.state.name == "PROCESSING":
            print('.', end='', flush=True)
            time.sleep(1)
            video_file = genai.get_file(video_file.name)

        if video_file.state.name == "FAILED":
            raise ValueError(f"Video upload failed: {video_name}")

        # Generate content with the prompt
        response = genai.GenerativeModel(model_name="gemini-1.5-pro-latest").generate_content(
            [video_file.uri, prompt_template],
            request_options={"timeout": 20}
        )

        # Print raw response for debugging
        #print(f"Raw response for {video_name}: {response.text}")

        # Save the response as a plain text file
        report_file_path = os.path.join(reports_folder, f"{video_name}_report.txt")
        with open(report_file_path, mode='w', encoding='utf-8') as report_file:
            report_file.write(response.text)
        print(f"Report saved for {video_name}: {report_file_path}")

    except Exception as e:
        print(f"Error processing {video_name}: {e}")


Processing video: Abuse012_x264.mp4
Uploaded: https://generativelanguage.googleapis.com/v1beta/files/wsbjctbkn52b
.Report saved for Abuse012_x264.mp4: /content/reports/Abuse012_x264.mp4_report.txt
Processing video: Abuse011_x264.mp4
Uploaded: https://generativelanguage.googleapis.com/v1beta/files/97rvdpvwd578
.Report saved for Abuse011_x264.mp4: /content/reports/Abuse011_x264.mp4_report.txt
Processing video: Abuse028_x264.mp4
Uploaded: https://generativelanguage.googleapis.com/v1beta/files/ny7az561fhes
..Report saved for Abuse028_x264.mp4: /content/reports/Abuse028_x264.mp4_report.txt
Processing video: Abuse008_x264.mp4
Uploaded: https://generativelanguage.googleapis.com/v1beta/files/es7wvsgaxpky
...Error processing Abuse008_x264.mp4: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Processing video: Abuse017_x264.mp4
Uploaded: https://generativelanguage.googleapis.com/v1beta/files/jceb8i5v1vv2
........Report saved for Abuse017_x264.mp4: /con

Error processing Abuse001_x264.mp4: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-pro-latest:generateContent?%24alt=json%3Benum-encoding%3Dint: Resource has been exhausted (e.g. check quota).
Processing video: Abuse044_x264.mp4
Uploaded: https://generativelanguage.googleapis.com/v1beta/files/vdda5l98qrb3
....Report saved for Abuse044_x264.mp4: /content/reports/Abuse044_x264.mp4_report.txt
Processing video: Abuse018_x264.mp4
Uploaded: https://generativelanguage.googleapis.com/v1beta/files/iyccq4r0of07
..Report saved for Abuse018_x264.mp4: /content/reports/Abuse018_x264.mp4_report.txt
Processing video: Abuse030_x264.mp4
Uploaded: https://generativelanguage.googleapis.com/v1beta/files/fttxlq0vlk3q
..Report saved for Abuse030_x264.mp4: /content/reports/Abuse030_x264.mp4_report.txt
Processing video: Abuse003_x264.mp4
Uploaded: https://generativelanguage.googleapis.com/v1beta/files/epa9u00o7wh
...Report saved for Abuse003_x264.mp4: /content/reports/Abuse003_x264

Error processing Abuse009_x264.mp4: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-pro-latest:generateContent?%24alt=json%3Benum-encoding%3Dint: Resource has been exhausted (e.g. check quota).
Processing video: Abuse023_x264.mp4
Uploaded: https://generativelanguage.googleapis.com/v1beta/files/76eqriy8ryq
..Report saved for Abuse023_x264.mp4: /content/reports/Abuse023_x264.mp4_report.txt
Processing video: Abuse002_x264.mp4
Uploaded: https://generativelanguage.googleapis.com/v1beta/files/3lom132iygn4
.Report saved for Abuse002_x264.mp4: /content/reports/Abuse002_x264.mp4_report.txt
Processing video: Arrest025_x264.mp4
Uploaded: https://generativelanguage.googleapis.com/v1beta/files/87annyt9xtvc
..Report saved for Arrest025_x264.mp4: /content/reports/Arrest025_x264.mp4_report.txt
Processing video: Abuse034_x264.mp4
Uploaded: https://generativelanguage.googleapis.com/v1beta/files/jv5nfc8cp2l1
..Report saved for Abuse034_x264.mp4: /content/reports/Abuse034_x264.

Error processing Arrest016_x264.mp4: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-pro-latest:generateContent?%24alt=json%3Benum-encoding%3Dint: Resource has been exhausted (e.g. check quota).
Processing video: Arrest009_x264.mp4
Uploaded: https://generativelanguage.googleapis.com/v1beta/files/gatnrdmbced
..Report saved for Arrest009_x264.mp4: /content/reports/Arrest009_x264.mp4_report.txt
Processing video: Arrest023_x264.mp4
Uploaded: https://generativelanguage.googleapis.com/v1beta/files/3odfrzhr0yhi
..Report saved for Arrest023_x264.mp4: /content/reports/Arrest023_x264.mp4_report.txt
Processing video: Arrest006_x264.mp4
Uploaded: https://generativelanguage.googleapis.com/v1beta/files/p8p7k0p90gds
.Report saved for Arrest006_x264.mp4: /content/reports/Arrest006_x264.mp4_report.txt
Processing video: Arrest014_x264.mp4
Uploaded: https://generativelanguage.googleapis.com/v1beta/files/k8dy1m5q667t
..

Error processing Arrest014_x264.mp4: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-pro-latest:generateContent?%24alt=json%3Benum-encoding%3Dint: Resource has been exhausted (e.g. check quota).
Processing video: Arrest028_x264.mp4
Uploaded: https://generativelanguage.googleapis.com/v1beta/files/9atvf2jg8185
...Report saved for Arrest028_x264.mp4: /content/reports/Arrest028_x264.mp4_report.txt
Processing video: Arrest011_x264.mp4
Uploaded: https://generativelanguage.googleapis.com/v1beta/files/1k63wrcbcocd
..

...

ERROR:tornado.access:503 POST /v1beta/models/gemini-1.5-pro-latest:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 4923.45ms


Report saved for Arrest015_x264.mp4: /content/reports/Arrest015_x264.mp4_report.txt
Processing video: Arrest013_x264.mp4
Uploaded: https://generativelanguage.googleapis.com/v1beta/files/hqm08an3004t
.

Error processing Arrest013_x264.mp4: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-pro-latest:generateContent?%24alt=json%3Benum-encoding%3Dint: Resource has been exhausted (e.g. check quota).
Processing video: Arrest003_x264.mp4
Uploaded: https://generativelanguage.googleapis.com/v1beta/files/v40odjzjr9q2
..

Error processing Arrest003_x264.mp4: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-pro-latest:generateContent?%24alt=json%3Benum-encoding%3Dint: Resource has been exhausted (e.g. check quota).
Processing video: Arrest022_x264.mp4
Uploaded: https://generativelanguage.googleapis.com/v1beta/files/c5866vddolga
..

KeyboardInterrupt: 

In [ ]:
# Zip the folder
!zip -r your_folder_name.zip /content/reports

# Download the zip file
from google.colab import files
files.download('reports.zip')


  adding: content/reports/ (stored 0%)
  adding: content/reports/Abuse002_x264.mp4_report.txt (deflated 38%)
  adding: content/reports/Arrest008_x264.mp4_report.txt (deflated 34%)
  adding: content/reports/Arrest018_x264.mp4_report.txt (deflated 43%)
  adding: content/reports/Abuse024_x264.mp4_report.txt (deflated 54%)
  adding: content/reports/Abuse043_x264.mp4_report.txt (deflated 34%)
  adding: content/reports/Arrest021_x264.mp4_report.txt (deflated 31%)
  adding: content/reports/Abuse044_x264.mp4_report.txt (deflated 33%)
  adding: content/reports/Abuse021_x264.mp4_report.txt (deflated 37%)
  adding: content/reports/Abuse036_x264.mp4_report.txt (deflated 36%)
  adding: content/reports/Arrest004_x264.mp4_report.txt (deflated 41%)
  adding: content/reports/Arrest026_x264.mp4_report.txt (deflated 36%)
  adding: content/reports/Abuse020_x264.mp4_report.txt (deflated 37%)
  adding: content/reports/Abuse047_x264.mp4_report.txt (deflated 36%)
  adding: content/reports/Arrest015_x264.mp4_r

FileNotFoundError: Cannot find file: reports.zip